In [15]:
import gurobipy as gp
from gurobipy import GRB
import math

In [2]:
def calculate_optimal_gurobi(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=100):
    m = gp.Model('box_constraint_test')
    m.setParam( 'OutputFlag', False )
    # Variables
    Bi = m.addVar(lb=lb, ub=ub, vtype=GRB.CONTINUOUS, name="Bi")
    Bi_supp = m.addVar(vtype=GRB.BINARY, name = '1[Bi]')
    Bi_sign = m.addVar(vtype=GRB.BINARY)
    Bi_pos = m.addVar(lb=0, ub=ub, vtype=GRB.CONTINUOUS)
    Bi_neg = m.addVar(lb=0, ub=-lb, vtype=GRB.CONTINUOUS)
    
    #Constraints
    m.addConstr(Bi_pos <= M*Bi_sign)
    m.addConstr(Bi_neg <= M*(1-Bi_sign))
    m.addConstr(Bi == Bi_pos - Bi_neg)

    m.addConstr(Bi >= -M*Bi_supp)
    m.addConstr(Bi <= M*Bi_supp)
    
    #Cost
    m.setObjective((B - Bi)*(B - Bi) + l0*Bi_supp + l1*(Bi_pos + Bi_neg) + l2*Bi*Bi, GRB.MINIMIZE)
    
    m.optimize()
    
    for v in m.getVars():
        if v.varName == "Bi":
            return v.x

In [68]:
def box(x, lb, ub):
    if x > ub:
        return ub
    elif x <= lb:
        return lb
    else:
        return x
    
def sign(x):
    if x < 0:
        return -1
    return 1

def calculate_closed_from(l0=None, l1=None, l2=None, B=None, lb=None, ub=None, M=None):
    Bi = B # Attempt to make Bi, B. This is solved for iteratively using other approaches
    
    x = box(sign(Bi)*(abs(Bi) - l1)/(1 + 2*l2), lb, ub)
    
    #print(x)
    
    condition1_for_zero_Bi = (abs(Bi) - l1)/(1 + 2*l2) < math.sqrt(2*l0/(1+2*l2))
    
    #print(condition1_for_zero_Bi)
    if condition1_for_zero_Bi:
        return 0
    
    condition2a_for_zero_Bi = not condition1_for_zero_Bi
    
    delta = (abs(Bi) - l1)**2 - 2*l0*(1 + 2*l2)
    
     
    x_range_low = ((abs(Bi) - l1)*sign(Bi) - math.sqrt(delta))/(1 + 2*l2) 
    x_range_high = (((abs(Bi) - l1)*sign(Bi) + math.sqrt(delta))/(1 + 2*l2))
    
    condition2b_for_zero_Bi = x_range_low <= x <= x_range_high
    
    if (condition2a_for_zero_Bi and condition2b_for_zero_Bi):
        return 0
    else:
        return x

In [81]:
def cost(B, Bi, l0, l1, l2):
    return (B - Bi)**2 + l0*(Bi!=0) + l1*abs(Bi) + l2*Bi**2

In [94]:
l0 = 2
l1 = 1
l2 = 1
B = 3
lb = -0.5
ub = 0.5

In [95]:
Bi_optimal = calculate_optimal(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_optimal, cost(B, Bi_optimal, l0, l1, l2))

0.5 9.0


In [96]:
Bi_box = calculate_closed_from(l0=l0, l1=l1, l2=l2, B=B , lb=lb, ub=ub)
print(Bi_box, cost(B, Bi_box, l0, l1, l2))

0 9
